In [1]:
#Importing libraries

import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import warnings
warnings.filterwarnings( 'ignore' )
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Function definition

def distribution_plot(df,column):
    plt.title('Distribution of ' + column)
    sns.distplot(df[column])
    plt.show()
    return


In [3]:
#Loading data
customers = pd.read_csv("datas/olist_customers_dataset.csv")
order_items = pd.read_csv("datas/olist_order_items_dataset.csv")
order_payments = pd.read_csv("datas/olist_order_payments_dataset.csv")
orders = pd.read_csv("datas/olist_orders_dataset.csv")

In [6]:
data = {
    'Customers': customers,
    'Order Items': order_items,
    'Payments': order_payments,
    'Orders': orders
}
#Datasets shapes
for x,y in data.items():
    print(x,y.shape)

Customers (99441, 5)
Order Items (112650, 7)
Payments (103886, 5)
Orders (99441, 8)


In [7]:
#Finding NaN values
for x,y in data.items():
    print(x,'\n')
    print(y.isna().sum(),'\n')

Customers 

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

Order Items 

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64 

Payments 

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64 

Orders 

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64 



In [8]:
#Finding duplicates
for x,y in data.items():
    print(x,'\n')
    print(y.duplicated().sum(),'\n')

Customers 

0 

Order Items 

0 

Payments 

0 

Orders 

0 



In [ ]:
#Merge the datasets into one dataframe
df = pd.merge(orders, order_payments, on='order_id')
df = df.merge(customers, on='customer_id')
df = df.merge(order_items, on='order_id')
df.head()

In [ ]:
#Summary statistics of the dataframe
df.describe()